In [8]:
import os
import numpy as np
import cv2

In [ ]:
#Read in images from folder
DATA_PATH = "/Users/sreeharirammohan/Desktop/Lesions"

classes = os.listdir(DATA_PATH)
del classes[0] #first entry is .DS Store not a directory

print(classes)

for class_name in classes:
    class_folder = os.path.join(DATA_PATH,class_name)

    sub_files = os.listdir(class_folder)
    
    for file in sub_files:
        if '.jpg' in file:
            image_filepath = os.path.join(class_folder, file)
            image_as_np_array = cv2.imread(image_filepath,flags=cv2.IMREAD_COLOR)
        


['Actinic Keratosis', 'Dermatofibroma']
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
